Reference:
1. Titanic: Feature Engineering & Logistic Regression - https://www.kaggle.com/matheuscoradini/titanic-feature-engineering-logistic-regression
2. Python Logistic regression - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
def age_nan(df):
    for i in df.Sex.unique():
        for j in df.Pclass.unique():
            x = df.loc[((df.Sex == i) & (df.Pclass == j)), 'Age'].mean()
            df.loc[((df.Sex == i) & (df.Pclass == j)), 'Age'] = df.loc[((df.Sex == i) & (df.Pclass == j)), 'Age'].fillna(x)

age_nan(train_data)
age_nan(test_data)

# fill NaN values of Embarked with 'S', because it's the most commom value for it

train_data['Embarked'] = train_data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')


train_data['Cabin_NaN'] = train_data['Cabin'].isnull().astype(int)
test_data['Cabin_NaN'] = test_data['Cabin'].isnull().astype(int)


test_data.Fare = test_data.Fare.fillna(-1)


In [ ]:
def is_female(x):
    if x == 'female':
        return 1
    else:
        return 0

train_data['Sex_bin'] = train_data['Sex'].map(is_female)
test_data['Sex_bin'] = test_data['Sex'].map(is_female)

In [ ]:
def embarked_s(x):
    if x == 'S':
        return 1
    else:
        return 0

train_data['Embarked_S'] = train_data['Embarked'].map(embarked_s)
test_data['Embarked_S'] = test_data['Embarked'].map(embarked_s)

def embarked_c(x):
    if x == 'C':
        return 1
    else:
        return 0
    
train_data['Embarked_C'] = train_data['Embarked'].map(embarked_c)
test_data['Embarked_C'] = test_data['Embarked'].map(embarked_c)

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
Sex_Count = train_data.groupby(["Survived","Sex"])["Sex"].count()#
Pclass = train_data.groupby(["Survived","Pclass"])["Pclass"].count()
# 3 class died a lot more
Age = train_data.groupby(["Survived","Age"])["Age"].count()
SibSp = train_data.groupby(["Survived","SibSp"])["SibSp"].count()
# 1 sibling have more chanvce to survive
Parch = train_data.groupby(["Survived","Parch"])["Parch"].count()
Ticket = train_data.groupby(["Survived","Ticket"])["Ticket"].count()
Fare = train_data.groupby(["Survived","Fare"])["Fare"].count()
Cabin = train_data.groupby(["Survived","Cabin"])["Cabin"].count()
Embarked = train_data.groupby(["Survived","Embarked"])["Embarked"].count()
# C died more, S died less

print(Sex_Count,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked)



In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ['Age', 'Sex_bin', 'Pclass', 'Fare', 'SibSp', 'Parch', 'Embarked_S','Embarked_C', 'Cabin_NaN']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

clf = LogisticRegression(random_state=0,max_iter=500).fit(X, y)
predictions = clf.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

#output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")